Compare WPC URMA sample with STID URMA sample. These files were locally collected so script needs to be redirected if used in the future. See Bruce V. at WPC for data. Produces zone-trimmed maps of WR and other CWAS, histograms of accumulated precip.

In [ ]:
from glob import glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import xesmf as xe
import regionmask
import cartopy
import cartopy.crs as ccrs
import geopandas as gpd

cwa = 'WESTUS'

In [ ]:
stid_urma_dir = '/scratch/general/lustre/u1070830/urma/'
wpc_urma_dir = '/scratch/general/lustre/u1070830/wpc/urma/'

In [ ]:
stid_urma_files = np.array(glob(stid_urma_dir + '*.grib2'))
wpc_urma_files = np.array(glob(wpc_urma_dir + '*.grib2'))

In [ ]:
stid_urma_times = np.array([f.split('.')[1] for f in stid_urma_files])
wpc_urma_times = np.array([f.split('_')[3].split('.')[0] for f in wpc_urma_files])

In [ ]:
stid_urma = xr.open_mfdataset(
    sorted(stid_urma_files[np.where(np.isin(stid_urma_times, wpc_urma_times))]), 
    engine='cfgrib', combine='nested', concat_dim='valid_time')

stid_urma

In [ ]:
wpc_urma = xr.open_mfdataset(
    sorted(wpc_urma_files[np.where(np.isin(wpc_urma_times, stid_urma_times))]),
    engine='cfgrib', combine='nested', concat_dim='valid_time')#.sel(x=stid_urma.x, y=stid_urma.y)

wpc_urma

In [ ]:
plotvar = wpc_urma['tp']

plotvar = xr.where(plotvar > 0.01*25.4, 1, 0).sum(dim='valid_time')/plotvar.valid_time.size
plotvar['longitude'] = plotvar['longitude'] - 360

# Masking with regionmask
lons, lats = plotvar.longitude, plotvar.latitude

geodir = '../forecast-zones/'
zones_shapefile = glob(geodir + '*.shp')[0]

# Read the shapefile
zones = gpd.read_file(zones_shapefile)

# Prune to Western Region using TZ
zones = zones.set_index('TIME_ZONE').loc[['M', 'Mm', 'm', 'MP', 'P']].reset_index()
cwas = zones.dissolve(by='CWA').reset_index()[['CWA', 'geometry']]
_cwas = cwas.copy()

if cwa == 'WESTUS':
    _cwas['CWA'] = 'WESTUS'
    _cwas = _cwas.dissolve(by='CWA').reset_index()
    bounds = _cwas.total_bounds
else:
    bounds = _cwas[_cwas['CWA'] == cwa].bounds.values[0]
        
mask = regionmask.mask_3D_geopandas(_cwas, lons, lats).rename({'region':'cwa'})
mask['cwa'] = _cwas.iloc[mask.cwa]['CWA'].values.astype(str)
mask = mask.sel(cwa=cwa)

plotvar['x'] = mask.x
plotvar['y'] = mask.y

plotvar_masked = xr.where(mask, plotvar, np.nan)

In [ ]:
# Make the plot
plt.rcParams.update({'font.size': 18})
fig = plt.figure(figsize=(12, 12), facecolor='w')
ax = fig.add_axes([0, 0, 1, 1], projection=ccrs.PlateCarree())

zones.geometry.boundary.plot(color=None, linestyle='--', edgecolor='black', 
                             linewidth=0.75, ax=ax)

cwas.geometry.boundary.plot(color=None, edgecolor='black', linewidth=1.5, ax=ax)
ax.add_feature(cartopy.feature.OCEAN, zorder=100, color='w', edgecolor=None)
ax.coastlines(linewidth=2, zorder=101)

threshold = 0.01
start_date, end_date = wpc_urma.valid_time[0].values, wpc_urma.valid_time[-1].values

ax.set_title('WPC URMA\nObserved Relative Frequency (QPE > %0.2f") %2.1f%%\n'%(threshold, plotvar.mean()*100) + 
             '%d 6h Periods\n%s - %s'%(mask.shape[0], start_date, end_date))

cbd = ax.contourf(plotvar_masked.lon, plotvar_masked.lat, plotvar_masked,
           levels=np.arange(0, 0.61, .1))#plotvar.max().round(1)+.2, .1))

#[left, bottom, width, height]
cbar_ax = fig.add_axes([1.005, .075, .05, .85])
cbar = plt.colorbar(cbd, cax=cbar_ax)
cbar.ax.tick_params(labelsize=16)

plt.show()
print()

In [ ]:
plotvar = wpc_urma['tp']

plotvar['longitude'] = plotvar['longitude'] - 360
lons, lats = plotvar.longitude, plotvar.latitude

geodir = '../forecast-zones/'
zones_shapefile = glob(geodir + '*.shp')[0]
zones = gpd.read_file(zones_shapefile)
zones = zones.set_index('TIME_ZONE').loc[['M', 'Mm', 'm', 'MP', 'P']].reset_index()
cwas = zones.dissolve(by='CWA').reset_index()[['CWA', 'geometry']]
_cwas = cwas.copy()

if cwa == 'WESTUS':
    _cwas['CWA'] = 'WESTUS'
    _cwas = _cwas.dissolve(by='CWA').reset_index()
    bounds = _cwas.total_bounds
else:
    bounds = _cwas[_cwas['CWA'] == cwa].bounds.values[0]
        
mask = regionmask.mask_3D_geopandas(_cwas, lons, lats).rename({'region':'cwa'})
mask['cwa'] = _cwas.iloc[mask.cwa]['CWA'].values.astype(str)
mask = mask.sel(cwa=cwa)
plotvar['x'] = mask.x
plotvar['y'] = mask.y

wpc_urma_tp_masked = xr.where(mask, plotvar, np.nan)
wpc_urma_tp_masked_flat = wpc_urma_tp_masked.values.flatten()

plotvar = stid_urma['tp']

plotvar['longitude'] = plotvar['longitude'] - 360
lons, lats = plotvar.longitude, plotvar.latitude
        
mask = regionmask.mask_3D_geopandas(_cwas, lons, lats).rename({'region':'cwa'})
mask['cwa'] = _cwas.iloc[mask.cwa]['CWA'].values.astype(str)
mask = mask.sel(cwa=cwa)
plotvar['x'] = mask.x
plotvar['y'] = mask.y

stid_urma_tp_masked = xr.where(mask, plotvar, np.nan)
stid_urma_tp_masked_flat = stid_urma_tp_masked.values.flatten()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 10), facecolor='w')

bins = [0.01, 0.1, 0.25, 0.50, 0.75, 1.0, 1.25, 1.5, 2.0, 3.0, 4.0, 5.0]

plt.rcParams.update({'font.size': 12})
ax.hist(stid_urma_tp_masked_flat/25.4, bins=bins, edgecolor='r', density=True, histtype='step', linewidth=2, label='STID URMA')
ax.hist(wpc_urma_tp_masked_flat/25.4, bins=bins, edgecolor='k', density=True, histtype='step', linewidth=2, label='WPC URMA')

ax.set_xticks(bins)
ax.set_xticklabels(bins, rotation=70)

ax.set_yscale('log')

ax.legend(fontsize=18)
ax.grid()

ax.set_title('URMA\n QPE ' + 
             '%d 6h Periods\n%s - %s'%(mask.shape[0], start_date, end_date))

plt.show()